In [12]:
%run variables.ipynb

In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set() # Ajusta valor padrão de tema, escala e paleta de cor

In [14]:
df_country_codes = pd.read_excel('../raw_databases/country_codes_independence_status.xlsx')
df_country_codes = df_country_codes.iloc[:, :6]
df_country_codes.rename(columns={'Alpha-3 code': 'Code'}, inplace=True)
df_country_codes.to_csv('../outputs/country_info.csv')

df_country_codes

,Numeric code,English short name (using title case),Independent,Code,Sink OFC,Conduit OFC
0,4,Afghanistan,Yes,AFG,No,No
1,248,Åland Islands,No,ALA,No,No
2,8,Albania,Yes,ALB,No,No
3,12,Algeria,Yes,DZA,No,No
4,16,American Samoa,No,ASM,No,No
...,...,...,...,...,...,...
244,876,Wallis and Futuna,No,WLF,No,No
245,732,Western Sahara,No,ESH,No,No
246,887,Yemen,Yes,YEM,No,No
247,894,Zambia,Yes,ZMB,No,No


In [15]:
df_pop_age = pd.read_csv('../raw_databases/population-by-broad-age-group.csv', sep=',')
df_pop_age = df_pop_age[df_pop_age['Year'] >= 1996]
df_pop_age['Population 25+'] = df_pop_age.iloc[:, 6] + df_pop_age.iloc[:, 7]
df_pop_age['Total'] = df_pop_age.iloc[:, 3:8].sum(axis=1)

df_pop_age

,Entity,Code,Year,"Estimates, 1950 - 2020: Total population by broad age group, both sexes combined (thousands) - Population under age 5","Estimates, 1950 - 2020: Total population by broad age group, both sexes combined (thousands) - Population aged 5-14","Estimates, 1950 - 2020: Total population by broad age group, both sexes combined (thousands) - Population aged 15-24","Estimates, 1950 - 2020: Total population by broad age group, both sexes combined (thousands) - Population aged 25-64","Estimates, 1950 - 2020: Total population by broad age group, both sexes combined (thousands) - Population aged 65 or over",Population 25+,Total
46,Afghanistan,AFG,1996,3700.626,5362.522,3622.673,5720.931,446.692,6167.623,18853.444
47,Afghanistan,AFG,1997,3838.877,5517.718,3699.161,5844.763,456.607,6301.370,19357.126
48,Afghanistan,AFG,1998,3936.710,5646.924,3757.508,5934.454,462.174,6396.628,19737.770
49,Afghanistan,AFG,1999,4031.994,5796.719,3832.473,6041.825,467.836,6509.661,20170.847
50,Afghanistan,AFG,2000,4166.247,5993.773,3946.244,6197.428,476.265,6673.693,20779.957
...,...,...,...,...,...,...,...,...,...,...
17461,Zimbabwe,ZWE,2016,2279.248,3706.669,2827.750,4815.110,401.561,5216.671,14030.338
17462,Zimbabwe,ZWE,2017,2252.615,3811.729,2847.000,4912.888,412.367,5325.255,14236.599
17463,Zimbabwe,ZWE,2018,2196.377,3925.750,2887.057,5005.167,424.461,5429.628,14438.812
17464,Zimbabwe,ZWE,2019,2138.020,4036.215,2944.880,5089.825,436.533,5526.358,14645.473


In [16]:
df_pop_code = df_country_codes.merge(df_pop_age, on='Code')
df_pop_code = df_pop_code.iloc[:, [0,6,3,7,13,14]]
df_pop_code.to_csv('../outputs/over_25_population.csv')

df_pop_code

,Numeric code,Entity,Code,Year,Population 25+,Total
0,4,Afghanistan,AFG,1996,6167.623,18853.444
1,4,Afghanistan,AFG,1997,6301.370,19357.126
2,4,Afghanistan,AFG,1998,6396.628,19737.770
3,4,Afghanistan,AFG,1999,6509.661,20170.847
4,4,Afghanistan,AFG,2000,6673.693,20779.957
...,...,...,...,...,...,...
4995,716,Zimbabwe,ZWE,2016,5216.671,14030.338
4996,716,Zimbabwe,ZWE,2017,5325.255,14236.599
4997,716,Zimbabwe,ZWE,2018,5429.628,14438.812
4998,716,Zimbabwe,ZWE,2019,5526.358,14645.473


In [17]:
df_pop = pd.read_excel('../raw_databases/population_1950-2019.xlsx', sheet_name='ESTIMATES', skiprows=16, index_col=0)
lines_to_remove = list(range(1,27)) + [47,57,63,81,82,90,109,110,116,126,127,136,148,149,178,187,202,205,206,212,220,230,231,232,243,257,274,284]

filter = ~df_pop.index.isin(lines_to_remove)
df_pop = df_pop.loc[filter, :]
df_pop.rename(columns={'Region, subregion, country or area *': 'Country'}, inplace=True)
df_pop = df_pop.drop(['Variant', 'Notes', 'Type', 'Parent code'], axis = 1)
df_pop_stacked = pd.melt(df_pop, id_vars=['Country', 'Country code'], var_name="Year", value_name="Population").sort_values(by=["Country", "Year"])
df_pop_stacked['Year'] = df_pop_stacked['Year'].astype('int')
df_pop = df_pop_stacked.iloc[:, [1,0,2,3]]
df_pop.to_csv('../outputs/total_population.csv')

df_pop

# Dados sobre vaticano, pois estão faltando no df df_pop_code
# df_holy = df_pop[df_pop['Country'] == 'Holy See']
# columns = [1,3] + list(range(52,77))
# df_holy = df_holy.iloc[:, columns]
# df_holy = df_holy.melt(id_vars=['Country', 'Country code'], var_name='Year', value_name='Population +25')
# df_holy['Total'] = df_holy['Population +25']
# df_holy['Year'] = df_holy['Year'].astype('int')
# df_holy['Independent'] = 'Yes'
# df_holy.rename(columns={'Country': 'Entity', 'Country code': 'Numeric code'}, inplace=True)
# df_holy = df_holy.iloc[:, [1,0,5,2,3,4]]

# Using df_pop_code instead of df_pop by replacing variable
# df_pop = df_pop_code.append(other=df_holy, ignore_index=True)
# df_pop.rename(columns={'Total': 'Population Total', 'Numeric code': 'Country code'}, inplace=True)
# df_pop

,Country code,Country,Year,Population
81,4,Afghanistan,1950,7752.12
316,4,Afghanistan,1951,7840.15
551,4,Afghanistan,1952,7936
786,4,Afghanistan,1953,8039.68
1021,4,Afghanistan,1954,8151.32
...,...,...,...,...
15529,716,Zimbabwe,2016,14030.3
15764,716,Zimbabwe,2017,14236.6
15999,716,Zimbabwe,2018,14438.8
16234,716,Zimbabwe,2019,14645.5


In [18]:
df_schooling_gdp = pd.read_csv('../raw_databases/average-years-of-schooling-vs-gdp-per-capita.csv', sep=",", decimal='.')
df_schooling_gdp.rename(columns={'GDP per capita, PPP (constant 2011 international $)': 'GDP per capita (World Bank)', 'Average Total Years of Schooling for Adult Population (Lee-Lee (2016), Barro-Lee (2018) and UNDP (2018))': 'Avg. Adult Schooling', 'Population, total': 'Population'}, inplace=True)

print (df_schooling_gdp)

filter = (~df_schooling_gdp['Avg. Adult Schooling'].isna()) & (df_schooling_gdp['Year'] >= 1996)

# It is possible to have estimates for Population by adding 'Population' as an argument in the list below
df_schooling = df_schooling_gdp[filter][['Code','Entity', 'Year', 'Avg. Adult Schooling', 'GDP per capita (World Bank)', 'Population']]

df_schooling = df_country_codes.merge(df_schooling, on='Code').iloc[:,[0,2,4,5,6,7,8,9]]
df_schooling.rename(columns={'Numeric code': 'Country code'}, inplace=True)
df_schooling_new = df_schooling.set_index('Country code')
df_schooling_new = df_schooling_new[['Entity','Year','Avg. Adult Schooling','GDP per capita (World Bank)']]
# df_schooling_new.to_csv('../outputs/avg_adult_schooling_and_gdp_1996-2017.csv')

df_schooling_2018 = pd.read_excel('../raw_databases/Mean_years_of_schooling_2018.xlsx')
df_schooling_2018 = df_schooling_2018.melt(id_vars=['Country code','Country'], var_name='Year', value_name='Avg. Adult Schooling')
df_schooling_2018['GDP per capita (World Bank)'] = np.nan
df_schooling_2018.rename(columns={'Country': 'Entity'}, inplace=True)
df_schooling_2018.set_index('Country code', inplace=True)

df_schooling_updated = df_schooling_new.append(df_schooling_2018)
df_schooling_updated.sort_values(by=['Country code','Year'], inplace=True)

df_schooling_updated.to_csv('../outputs/avg_adult_schooling_and_gdp_1996-2017.csv')

df_schooling_updated


              Entity      Code  Year Continent  GDP per capita (World Bank)  \
0           Abkhazia  OWID_ABK  2015      Asia                          NaN   
1        Afghanistan       AFG  1870       NaN                          NaN   
2        Afghanistan       AFG  1875       NaN                          NaN   
3        Afghanistan       AFG  1880       NaN                          NaN   
4        Afghanistan       AFG  1885       NaN                          NaN   
...              ...       ...   ...       ...                          ...   
17341       Zimbabwe       ZWE  2014       NaN                  1925.138698   
17342       Zimbabwe       ZWE  2015    Africa                  1912.280261   
17343       Zimbabwe       ZWE  2016       NaN                  1879.628119   
17344       Zimbabwe       ZWE  2017       NaN                  1899.774977   
17345  Åland Islands       ALA  2015    Europe                          NaN   

       Population  Avg. Adult Schooling  
0        

,Entity,Year,Avg. Adult Schooling,GDP per capita (World Bank)
Country code,,,,
4,Afghanistan,1996,1.9,NaN
4,Afghanistan,1997,2.0,NaN
4,Afghanistan,1998,2.0,NaN
4,Afghanistan,1999,2.1,NaN
4,Afghanistan,2000,2.2,NaN
...,...,...,...,...
894,Zambia,2014,6.9,3632.503753
894,Zambia,2015,6.9,3627.202041
894,Zambia,2016,7.0,3652.151926


In [19]:
# Loading GDP per capita data from Penn World Tables (PWT)
df_gdp = pd.read_csv('../raw_databases/real-gdp-per-capita-PWT.csv', sep=",", decimal='.')
df_gdp.rename(columns={'Output-side real GDP per capita (gdppc_o) (PWT 9.1 (2019))': 'GDP per capita (PWT)'}, inplace=True)

print(df_gdp)

df_gdp = pd.merge(df_country_codes, df_gdp, on='Code')
df_gdp = pd.DataFrame(df_gdp.set_index('Numeric code').loc[:, ['Entity', 'Code', 'Year', 'GDP per capita (PWT)']])
df_gdp.to_csv('../outputs/gdp_per_capita_PWT.csv')

df_gdp

        Entity Code  Year  GDP per capita (PWT)
0      Albania  ALB  1970             2828.0728
1      Albania  ALB  1971             2869.7212
2      Albania  ALB  1972             2919.5935
3      Albania  ALB  1973             2968.1025
4      Albania  ALB  1974             3024.6765
...        ...  ...   ...                   ...
9980  Zimbabwe  ZWE  2013             1881.8159
9981  Zimbabwe  ZWE  2014             1904.7742
9982  Zimbabwe  ZWE  2015             1847.6211
9983  Zimbabwe  ZWE  2016             1821.6588
9984  Zimbabwe  ZWE  2017             1871.8087

[9985 rows x 4 columns]


,Entity,Code,Year,GDP per capita (PWT)
Numeric code,,,,
8,Albania,ALB,1970,2828.0728
8,Albania,ALB,1971,2869.7212
8,Albania,ALB,1972,2919.5935
8,Albania,ALB,1973,2968.1025
8,Albania,ALB,1974,3024.6765
...,...,...,...,...
716,Zimbabwe,ZWE,2013,1881.8159
716,Zimbabwe,ZWE,2014,1904.7742
716,Zimbabwe,ZWE,2015,1847.6211


In [20]:
df_gov = pd.read_excel('../raw_databases/wgidataset.xlsx', sheet_name='GovernmentEffectiveness', skiprows=13, header=[0,1])
df_gov.rename(columns={'Unnamed: 0_level_0': 'Country', 'Unnamed: 1_level_0': 'Code'}, inplace=True)


# df_gov_est = df_gov.xs(('Estimate'), level=1, axis=1, drop_level=True)
df_gov_est = df_gov.loc[:, (slice(None), ['Country/Territory', 'Code', 'Estimate'])]
df_gov_est.columns = df_gov_est.columns.droplevel(1)


df_gov = df_gov_est.melt(id_vars=['Country', 'Code'], var_name='Year', value_name='Gov. Effectiveness').sort_values(by=['Country', 'Year'])
df_gov = pd.merge(df_country_codes[['Numeric code', 'Code']], df_gov, on='Code')
df_gov = df_gov.rename(columns={"Numeric code": "Country code"}).set_index('Country code')
df_gov.to_csv('../outputs/gov_effectiveness.csv')

df_gov

,Code,Country,Year,Gov. Effectiveness
Country code,,,,
4,AFG,Afghanistan,1996,-2.175167
4,AFG,Afghanistan,1998,-2.135942
4,AFG,Afghanistan,2000,-2.231651
4,AFG,Afghanistan,2002,-1.567973
4,AFG,Afghanistan,2003,-1.082488
...,...,...,...,...
716,ZWE,Zimbabwe,2015,-1.157869
716,ZWE,Zimbabwe,2016,-1.158350
716,ZWE,Zimbabwe,2017,-1.188722


In [21]:
df_sci_tech = pd.read_excel('../raw_databases/Data_Extract_From_World_Development_Indicators.xlsx', skipfooter=757, na_values='..')
# df_sci_tech = pd.read_csv('raw_databases\Data_Extract_From_World_Development_Indicators.csv')
years = list(range(1990,2021))
new_columns = ['Country Name', 'Code', 'Series Name', 'Series Code'] + years
df_sci_tech.columns = new_columns
del df_sci_tech['Series Code']
df_sci_tech = df_sci_tech.pivot(index=('Code'), columns='Series Name', values=(years))
df_sci_tech.columns.names = ['Year', '']
df_sci_tech = df_sci_tech.stack(0).reset_index()
df_sci_tech = df_sci_tech[df_sci_tech['Code'] != 'XKX']

df_sci_tech_new = df_country_codes.merge(df_sci_tech, how='right')
df_sci_tech_new.set_index('Numeric code', inplace=True)
df_sci_tech_new = df_sci_tech_new.iloc[:, 5:]
df_sci_tech_new.to_csv('../outputs/sci_tech.csv')

df_sci_tech_new

,Year,"Charges for the use of intellectual property, payments (BoP, current US$)","Charges for the use of intellectual property, receipts (BoP, current US$)",Firms that spend on R&D (% of firms),High-technology exports (current US$),"Industrial design applications, nonresident, by count","Industrial design applications, resident, by count",Labor force with advanced education (% of total working-age population with advanced education),"Patent applications, nonresidents","Patent applications, residents",Research and development expenditure (% of GDP),Researchers in R&D (per million people),Scientific and technical journal articles,Technicians in R&D (per million people),"Trademark applications, nonresident, by count","Trademark applications, resident, by count","Trademark applications, total"
Numeric code,,,,,,,,,,,,,,,,,
4,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.00,NaN,NaN,NaN,NaN
4,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN
4,2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.50,NaN,NaN,NaN,NaN
4,2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.34,NaN,NaN,NaN,NaN
4,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.75,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716,2015,1.954557e+07,2.267903e+06,NaN,11830010.0,NaN,NaN,NaN,19.0,9.0,NaN,NaN,268.98,NaN,2399.0,285.0,1975.0
716,2016,1.037293e+07,2.372227e+06,12.4,13735165.0,NaN,NaN,NaN,5.0,8.0,NaN,NaN,313.80,NaN,3086.0,155.0,2021.0
716,2017,4.322232e+06,0.000000e+00,NaN,11206210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,340.45,NaN,NaN,NaN,NaN


In [22]:

path = '../subject_areas/All/scimagojr country rank 2017.xlsx'
df_scimago = pd.read_excel(path)
year = int(path[-9:-5])

country_codes = []

print(df_pop)
for index, row in df_scimago.iterrows():
    if (row['Country'] == df_pop.iloc[:, 1]).any():
        filter = (df_pop.iloc[:, 1] == row['Country']) & (df_pop['Year'] == year)
        country_codes.append(df_pop.loc[filter, "Country code"].item())
    else:
        print(f"Problem! {row['Country']} = {scimago_codes[row['Country']]}")
        country_codes.append(scimago_codes[row['Country']])

df_scimago["Country code"] = pd.Series(country_codes)
h_index = pd.DataFrame(df_scimago.set_index('Country code').loc[:, "H index"])

# Agrupamento por País-Pai; para ver territórios:  adicionar df_scimago['Country'] como argumento na lista do groupby()
df_scimago = df_scimago[['Documents', 'Citable documents', 'Citations', 'Self-citations']].groupby([df_scimago['Country code']]).sum()

df_scimago['Citations per Document'] = round((df_scimago['Citations'] / df_scimago['Documents']), 2)
df_scimago = df_scimago.join(h_index, how='left')

# For df_pop_code
# df_pop_new = df_pop.set_index('Country code')
# print(df_pop_new)
# df_pop_new = df_pop_new[df_pop_new['Year'] == year]
# print(df_pop_new)


# if (df_scimago.index.get_level_values(0) == 530).any():
#     pop_netherlands_antilles = round(df_pop_new[(df_pop_new.index == 531) | (df_pop_new.index == 534) | (df_pop_new.index == 533) | (df_pop_new.index == 535)][['Population +25','Population Total']].apply('sum'), 3)
#     df_pop_new.loc[530] = ["Netherlands Antilles", "No", year, pop_netherlands_antilles[0], pop_netherlands_antilles[1]]


# For df_pop
df_pop_indexed = df_pop.set_index('Country code')
df_pop_new = df_pop_indexed[df_pop_indexed['Year'] == year]
df_pop_new['Population 25 Years Before'] = df_pop_indexed[df_pop_indexed['Year'] == (year - 25)]['Population']
print(df_pop_new)

if (df_scimago.index.get_level_values(0) == 530).any():
    pop_netherlands_antilles = round(df_pop_new[(df_pop_new.index == 531) | (df_pop_new.index == 534) | (df_pop_new.index == 533) | (df_pop_new.index == 535)][['Population', 'Population 25 Years Before']].apply('sum'), 3)
    df_pop_new.loc[530] = ["Netherlands Antilles", year, pop_netherlands_antilles[0], pop_netherlands_antilles[1]]


df_pop_code.rename(columns={'Total': 'Population Total', 'Numeric code': 'Country code'}, inplace=True)
df_pop_code_new = df_pop_code.set_index('Country code')
df_pop_code_new = df_pop_code_new[df_pop_code_new['Year'] == year].iloc[:,[3,4]]
print(df_pop_code_new)
df_pop_new = df_pop_new.join(df_pop_code_new, how='left')
print(df_pop_new)



df_scimago_pop = df_pop_new.join(df_scimago, how='right')

# For df_pop_code
# df_scimago_pop['CPC'] = df_scimago_pop['Citable documents'] / df_scimago_pop['Population +25']

# For df_pop
df_scimago_pop[['Population', 'Population 25 Years Before']].astype('float') # = df_scimago_pop['Population'].astype('float')
print(df_scimago_pop)



# df_scimago_pop.describe(include='all')
# df_scimago_pop[df_scimago_pop['CPC'] == df_scimago_pop['CPC'].max()]
# df_scimago_pop[df_scimago_pop.index == 336]
# df_scimago_pop.sort_values(by='CPC', ascending=False)

# For df_pop_Code
# df_country_codes[df_country_codes["Numeric code"].isin(df_scimago_pop[df_scimago_pop['Entity'].isna()].index)]


       Country code      Country  Year Population
81                4  Afghanistan  1950    7752.12
316               4  Afghanistan  1951    7840.15
551               4  Afghanistan  1952       7936
786               4  Afghanistan  1953    8039.68
1021              4  Afghanistan  1954    8151.32
...             ...          ...   ...        ...
15529           716     Zimbabwe  2016    14030.3
15764           716     Zimbabwe  2017    14236.6
15999           716     Zimbabwe  2018    14438.8
16234           716     Zimbabwe  2019    14645.5
16469           716     Zimbabwe  2020    14862.9

[16685 rows x 4 columns]
Problem! United States = 840
Problem! South Korea = 410
Problem! Iran = 364
Problem! Taiwan = 158
Problem! Czech Republic = 203
Problem! Hong Kong = 344
Problem! Macao = 446
Problem! Venezuela = 862
Problem! Tanzania = 834
Problem! Macedonia = 807
Problem! Palestine = 275
Problem! Moldova = 498
Problem! Côte d’Ivoire = 384
Problem! Bolivia = 68
Problem! Laos = 418
Problem

In [23]:
df_schooling_new = df_schooling_new.loc[(df_schooling_new['Year'] == year), ['Avg. Adult Schooling','GDP per capita (World Bank)']]

df_sci_pop_schooling = df_scimago_pop.join(df_schooling_new, how='left')
df_country_independence = pd.DataFrame(df_country_codes.set_index('Numeric code').loc[:, ['Independent', 'Sink OFC', 'Conduit OFC', 'Code']])
df_sci_pop_schooling = df_country_independence.join(df_sci_pop_schooling, how='right')
df_sci_pop_schooling


,Independent,Sink OFC,Conduit OFC,Code,Country,Year,Population,Population 25 Years Before,Population 25+,Population Total,Documents,Citable documents,Citations,Self-citations,Citations per Document,H index,Avg. Adult Schooling,GDP per capita (World Bank)
4,Yes,No,No,AFG,Afghanistan,2017,36296.1,14485.5,12618.205,36296.111,198,179,4874,70,24.62,60,3.8,1803.987487
8,Yes,No,No,ALB,Albania,2017,2884.17,3245.89,1892.473,2884.169,403,371,2767,168,6.87,72,10.0,11803.430594
12,Yes,No,No,DZA,Algeria,2017,41389.2,27028.3,22879.494,41389.174,7419,7161,37843,7635,5.10,178,8.0,13913.839363
16,No,No,No,ASM,American Samoa,2017,55.617,49.9,NaN,NaN,11,11,54,6,4.91,29,NaN,NaN
20,Yes,Yes,No,AND,Andorra,2017,76.997,58.882,NaN,NaN,17,14,106,5,6.24,28,10.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860,Yes,No,No,UZB,Uzbekistan,2017,31959.8,21375.1,17145.037,31959.774,615,594,2651,359,4.31,94,11.5,6253.103860
862,Yes,No,No,VEN,Venezuela (Bolivarian Republic of),2017,29402.5,20557.7,16254.828,29402.480,1725,1561,7860,660,4.56,221,10.3,NaN
882,Yes,Yes,No,WSM,Samoa,2017,195.358,165.49,85.281,195.358,43,34,175,20,4.07,37,10.3,6021.557424
887,Yes,No,No,YEM,Yemen,2017,27834.8,12954.2,10745.291,27834.811,460,445,6995,385,15.21,78,3.0,NaN


It is increasingly being recognized that tax havens, or corporate tax havens, have distorted economic data which produces artificially high, or inflated, GDP-per-capita figures (How tax havens turn economic statistics into nonsense". Quartz. 11 June 2018.). In the figure below you can compare the estimates of GDP-per-capita from the World Bank and the Penn World Tables (PWT). As you can see, while most of the data from the two different sources converge, the World Bank data shows inflated values for known tax havens(i.e., Singapore, Macao, Luxembourg, Qatar, Hong Kong, Kuwait, Bermuda and Brunei)(Dharmapala, Dhammika; Hines, James R., Jr. (2009). "Which Countries Become Tax Havens?" (PDF). Journal of Public Economics. 93 (9–10): 1058–1068). Therefore, I have decided to use GDP-per-capita data from the PWT, as these have more conservative values for tax havens. Constructed by Robert Summers and Alan Heston of the University of Pennsylvania, together with the late Irving Kravis, the PWT are the definitive source for real national accounts data. The national accounts for each country, initially in their own currencies, are adjusted using detailed price data to obtain real national accounts in a common currency (U.S. dollars) across countries. These data are invaluable for making comparisons of Gross Domestic Product across countries, and for all manner of research on development and growth.

![World Bank vs. PWT](gdp-per-capita-inflation--and-ppp-adjusted-world-bank-data-vs-penn-world-table-data.png)

In [24]:
filter = (df_gdp['Year'] == year)
df_gdp = df_gdp.loc[filter, 'GDP per capita (PWT)']



df_sci_pop_schooling = df_sci_pop_schooling.join(df_gdp, how='left')


df_sci_pop_schooling['MAX TAMS'] = df_sci_pop_schooling['Avg. Adult Schooling'] * df_sci_pop_schooling['Population 25 Years Before']
df_sci_pop_schooling['REAL TAMS'] = df_sci_pop_schooling['Avg. Adult Schooling'] * df_sci_pop_schooling['Population 25+']
df_sci_pop_schooling['Documents / M-TAMS'] = df_sci_pop_schooling['Documents'] / df_sci_pop_schooling['MAX TAMS']
df_sci_pop_schooling['Documents / R-TAMS'] = df_sci_pop_schooling['Documents'] / df_sci_pop_schooling['REAL TAMS']
df_sci_pop_schooling['Citations / M-TAMS'] = df_sci_pop_schooling['Citations per Document'] * df_sci_pop_schooling['Documents / M-TAMS']
df_sci_pop_schooling['Citations / R-TAMS'] = df_sci_pop_schooling['Citations per Document'] * df_sci_pop_schooling['Documents / R-TAMS']


df_sci_pop_schooling

,Independent,Sink OFC,Conduit OFC,Code,Country,Year,Population,Population 25 Years Before,Population 25+,Population Total,...,H index,Avg. Adult Schooling,GDP per capita (World Bank),GDP per capita (PWT),MAX TAMS,REAL TAMS,Documents / M-TAMS,Documents / R-TAMS,Citations / M-TAMS,Citations / R-TAMS
4,Yes,No,No,AFG,Afghanistan,2017,36296.1,14485.5,12618.205,36296.111,...,60,3.8,1803.987487,NaN,55045.1,47949.1790,0.00359705,0.004129,0.0885594,0.101665
8,Yes,No,No,ALB,Albania,2017,2884.17,3245.89,1892.473,2884.169,...,72,10.0,11803.430594,11941.1970,32458.9,18924.7300,0.0124157,0.021295,0.0852959,0.146296
12,Yes,No,No,DZA,Algeria,2017,41389.2,27028.3,22879.494,41389.174,...,178,8.0,13913.839363,12323.5760,216227,183035.9520,0.0343112,0.040533,0.174987,0.206718
16,No,No,No,ASM,American Samoa,2017,55.617,49.9,NaN,NaN,...,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,Yes,Yes,No,AND,Andorra,2017,76.997,58.882,NaN,NaN,...,28,10.2,NaN,NaN,600.596,NaN,0.0283052,NaN,0.176624,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860,Yes,No,No,UZB,Uzbekistan,2017,31959.8,21375.1,17145.037,31959.774,...,94,11.5,6253.103860,9595.3623,245814,197167.9255,0.00250189,0.003119,0.0107832,0.013444
862,Yes,No,No,VEN,Venezuela (Bolivarian Republic of),2017,29402.5,20557.7,16254.828,29402.480,...,221,10.3,NaN,7696.6719,211744,167424.7284,0.00814662,0.010303,0.0371486,0.046982
882,Yes,Yes,No,WSM,Samoa,2017,195.358,165.49,85.281,195.358,...,37,10.3,6021.557424,NaN,1704.55,878.3943,0.0252266,0.048953,0.102672,0.199239
887,Yes,No,No,YEM,Yemen,2017,27834.8,12954.2,10745.291,27834.811,...,78,3.0,NaN,1850.2374,38862.5,32235.8730,0.0118366,0.014270,0.180035,0.217044


In [25]:

df_gov_new = df_gov[df_gov['Year'] == year]

df_sci_pop_schooling_gov = df_sci_pop_schooling.join(pd.DataFrame(df_gov_new['Gov. Effectiveness']), how='left')
df_sci_pop_schooling_gov


,Independent,Sink OFC,Conduit OFC,Code,Country,Year,Population,Population 25 Years Before,Population 25+,Population Total,...,Avg. Adult Schooling,GDP per capita (World Bank),GDP per capita (PWT),MAX TAMS,REAL TAMS,Documents / M-TAMS,Documents / R-TAMS,Citations / M-TAMS,Citations / R-TAMS,Gov. Effectiveness
4,Yes,No,No,AFG,Afghanistan,2017,36296.1,14485.5,12618.205,36296.111,...,3.8,1803.987487,NaN,55045.1,47949.1790,0.00359705,0.004129,0.0885594,0.101665,-1.334610
8,Yes,No,No,ALB,Albania,2017,2884.17,3245.89,1892.473,2884.169,...,10.0,11803.430594,11941.1970,32458.9,18924.7300,0.0124157,0.021295,0.0852959,0.146296,0.083774
12,Yes,No,No,DZA,Algeria,2017,41389.2,27028.3,22879.494,41389.174,...,8.0,13913.839363,12323.5760,216227,183035.9520,0.0343112,0.040533,0.174987,0.206718,-0.589159
16,No,No,No,ASM,American Samoa,2017,55.617,49.9,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.560554
20,Yes,Yes,No,AND,Andorra,2017,76.997,58.882,NaN,NaN,...,10.2,NaN,NaN,600.596,NaN,0.0283052,NaN,0.176624,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860,Yes,No,No,UZB,Uzbekistan,2017,31959.8,21375.1,17145.037,31959.774,...,11.5,6253.103860,9595.3623,245814,197167.9255,0.00250189,0.003119,0.0107832,0.013444,-0.557881
862,Yes,No,No,VEN,Venezuela (Bolivarian Republic of),2017,29402.5,20557.7,16254.828,29402.480,...,10.3,NaN,7696.6719,211744,167424.7284,0.00814662,0.010303,0.0371486,0.046982,-1.398328
882,Yes,Yes,No,WSM,Samoa,2017,195.358,165.49,85.281,195.358,...,10.3,6021.557424,NaN,1704.55,878.3943,0.0252266,0.048953,0.102672,0.199239,0.616182
887,Yes,No,No,YEM,Yemen,2017,27834.8,12954.2,10745.291,27834.811,...,3.0,NaN,1850.2374,38862.5,32235.8730,0.0118366,0.014270,0.180035,0.217044,-1.915434


In [26]:
df = pd.DataFrame()
db = {}

df_test = df_sci_pop_schooling_gov.to_dict()

db[year] = df_test

db[2017].keys()

df = df.append(df_sci_pop_schooling_gov)
df

,Independent,Sink OFC,Conduit OFC,Code,Country,Year,Population,Population 25 Years Before,Population 25+,Population Total,...,Avg. Adult Schooling,GDP per capita (World Bank),GDP per capita (PWT),MAX TAMS,REAL TAMS,Documents / M-TAMS,Documents / R-TAMS,Citations / M-TAMS,Citations / R-TAMS,Gov. Effectiveness
4,Yes,No,No,AFG,Afghanistan,2017,36296.1,14485.5,12618.205,36296.111,...,3.8,1803.987487,NaN,55045.1,47949.1790,0.00359705,0.004129,0.0885594,0.101665,-1.334610
8,Yes,No,No,ALB,Albania,2017,2884.17,3245.89,1892.473,2884.169,...,10.0,11803.430594,11941.1970,32458.9,18924.7300,0.0124157,0.021295,0.0852959,0.146296,0.083774
12,Yes,No,No,DZA,Algeria,2017,41389.2,27028.3,22879.494,41389.174,...,8.0,13913.839363,12323.5760,216227,183035.9520,0.0343112,0.040533,0.174987,0.206718,-0.589159
16,No,No,No,ASM,American Samoa,2017,55.617,49.9,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.560554
20,Yes,Yes,No,AND,Andorra,2017,76.997,58.882,NaN,NaN,...,10.2,NaN,NaN,600.596,NaN,0.0283052,NaN,0.176624,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860,Yes,No,No,UZB,Uzbekistan,2017,31959.8,21375.1,17145.037,31959.774,...,11.5,6253.103860,9595.3623,245814,197167.9255,0.00250189,0.003119,0.0107832,0.013444,-0.557881
862,Yes,No,No,VEN,Venezuela (Bolivarian Republic of),2017,29402.5,20557.7,16254.828,29402.480,...,10.3,NaN,7696.6719,211744,167424.7284,0.00814662,0.010303,0.0371486,0.046982,-1.398328
882,Yes,Yes,No,WSM,Samoa,2017,195.358,165.49,85.281,195.358,...,10.3,6021.557424,NaN,1704.55,878.3943,0.0252266,0.048953,0.102672,0.199239,0.616182
887,Yes,No,No,YEM,Yemen,2017,27834.8,12954.2,10745.291,27834.811,...,3.0,NaN,1850.2374,38862.5,32235.8730,0.0118366,0.014270,0.180035,0.217044,-1.915434
